In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix

BASE_DIR = Path.cwd().parents[0] if Path.cwd().name == "notebooks" else Path.cwd()
DATA_DIR = BASE_DIR / "data" / "final"

df = pd.read_csv(DATA_DIR / "brasileirao_final.csv")
print(df.shape)
df.head()


(8453, 41)


,ano_campeonato,data,rodada,estadio,arbitro,publico,publico_max,time_mandante,time_visitante,tecnico_mandante,...,chutes_mandante,chutes_visitante,chutes_fora_mandante,chutes_fora_visitante,resultado,pontos_mandante,pontos_visitante,mandante_venceu,visitante_venceu,empate_flag
0,2003,2003-05-04,1,Estádio Brinco de Ouro da Princesa,Não informado,12685.0,37182.0,Guarani,Vasco da Gama,Desconhecido,...,0.0,0.0,0.0,0.0,mandante,3,0,1,0,0
1,2003,2003-05-04,1,Estádio Governador Magalhães Pinto,Não informado,12685.0,37182.0,Cruzeiro,São Caetano,Desconhecido,...,0.0,0.0,0.0,0.0,empate,1,1,0,0,1
2,2003,2003-05-04,1,Estádio Jornalista Mário Filho,Não informado,12685.0,37182.0,Flamengo,Coritiba FC,Desconhecido,...,0.0,0.0,0.0,0.0,empate,1,1,0,0,1
3,2003,2003-05-04,1,Estádio Governador Plácido Castelo,Não informado,12685.0,37182.0,Fortaleza,EC Bahia,Desconhecido,...,0.0,0.0,0.0,0.0,empate,1,1,0,0,1
4,2003,2003-05-04,1,Estádio de Hailé Pinheiro,Não informado,12685.0,37182.0,Goiás EC,Paysandu SC,Desconhecido,...,0.0,0.0,0.0,0.0,empate,1,1,0,0,1


In [2]:
# Target (rótulo)
y = df["resultado"]  # valores: 'mandante', 'visitante', 'empate'

# Features numéricas que existem antes do jogo
features = [
    "ano_campeonato",
    "rodada",
    "colocacao_mandante",
    "colocacao_visitante",
    "valor_equipe_titular_mandante",
    "valor_equipe_titular_visitante",
    "idade_media_titular_mandante",
    "idade_media_titular_visitante",
    "publico_max",
]

X = df[features].copy()

# Tratar NaN nessas colunas: preencher com mediana da coluna
for col in features:
    X[col] = X[col].fillna(X[col].median())

# Remover linhas onde o resultado ainda é NaN (casos raros)
mask_valid = y.notna()
X = X[mask_valid]
y = y[mask_valid]

X.shape, y.shape


((8452, 9), (8452,))

In [3]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

X_train.shape, X_test.shape


((6761, 9), (1691, 9))

In [4]:
modelo = Pipeline(
    steps=[
        ("scaler", StandardScaler()),
        (
            "clf",
            LogisticRegression(
                multi_class="multinomial",
                max_iter=1000,
                n_jobs=-1,
            ),
        ),
    ]
)

modelo.fit(X_train, y_train)

y_pred = modelo.predict(X_test)

print("Relatório de classificação:\n")
print(classification_report(y_test, y_pred))

print("Matriz de confusão:\n")
print(confusion_matrix(y_test, y_pred))


/Users/fabrica/Documents/ia-futebol-brasil/.venv/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Relatório de classificação:

              precision    recall  f1-score   support

      empate       0.00      0.00      0.00       450
    mandante       0.52      0.93      0.67       837
   visitante       0.39      0.18      0.24       404

    accuracy                           0.51      1691
   macro avg       0.30      0.37      0.30      1691
weighted avg       0.35      0.51      0.39      1691

Matriz de confusão:

[[  0 393  57]
 [  0 782  55]
 [  0 332  72]]


/Users/fabrica/Documents/ia-futebol-brasil/.venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/fabrica/Documents/ia-futebol-brasil/.venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/fabrica/Documents/ia-futebol-brasil/.venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


In [5]:
from dataclasses import dataclass

@dataclass
class PartidaEntrada:
    ano_campeonato: int
    rodada: int
    colocacao_mandante: float
    colocacao_visitante: float
    valor_equipe_titular_mandante: float
    valor_equipe_titular_visitante: float
    idade_media_titular_mandante: float
    idade_media_titular_visitante: float
    publico_max: float


def prever_partida(partida: PartidaEntrada):
    """
    Recebe os dados de uma partida pré-jogo e retorna
    as probabilidades de vitória do mandante, empate e vitória do visitante.
    """
    dados = pd.DataFrame([{
        "ano_campeonato": partida.ano_campeonato,
        "rodada": partida.rodada,
        "colocacao_mandante": partida.colocacao_mandante,
        "colocacao_visitante": partida.colocacao_visitante,
        "valor_equipe_titular_mandante": partida.valor_equipe_titular_mandante,
        "valor_equipe_titular_visitante": partida.valor_equipe_titular_visitante,
        "idade_media_titular_mandante": partida.idade_media_titular_mandante,
        "idade_media_titular_visitante": partida.idade_media_titular_visitante,
        "publico_max": partida.publico_max,
    }])

    # Se faltar alguma coisa, preenche com mediana global (segurança extra)
    for col in features:
        if dados[col].isna().any():
            dados[col] = dados[col].fillna(X[col].median())

    probs = modelo.predict_proba(dados)[0]
    classes = modelo.classes_

    return dict(zip(classes, probs))


In [6]:
# Escolhe uma partida aleatória do dataset para teste
linha = df.sample(1, random_state=1).iloc[0]

entrada = PartidaEntrada(
    ano_campeonato=int(linha["ano_campeonato"]),
    rodada=int(linha["rodada"]),
    colocacao_mandante=float(linha["colocacao_mandante"]) if not np.isnan(linha["colocacao_mandante"]) else X["colocacao_mandante"].median(),
    colocacao_visitante=float(linha["colocacao_visitante"]) if not np.isnan(linha["colocacao_visitante"]) else X["colocacao_visitante"].median(),
    valor_equipe_titular_mandante=float(linha["valor_equipe_titular_mandante"]) if not np.isnan(linha["valor_equipe_titular_mandante"]) else X["valor_equipe_titular_mandante"].median(),
    valor_equipe_titular_visitante=float(linha["valor_equipe_titular_visitante"]) if not np.isnan(linha["valor_equipe_titular_visitante"]) else X["valor_equipe_titular_visitante"].median(),
    idade_media_titular_mandante=float(linha["idade_media_titular_mandante"]) if not np.isnan(linha["idade_media_titular_mandante"]) else X["idade_media_titular_mandante"].median(),
    idade_media_titular_visitante=float(linha["idade_media_titular_visitante"]) if not np.isnan(linha["idade_media_titular_visitante"]) else X["idade_media_titular_visitante"].median(),
    publico_max=float(linha["publico_max"]) if not np.isnan(linha["publico_max"]) else X["publico_max"].median(),
)

print("Resultado real dessa partida:", linha["resultado"])
print("Probabilidades previstas:")
prever_partida(entrada)


Resultado real dessa partida: empate
Probabilidades previstas:


{'empate': np.float64(0.2452528097697559),
 'mandante': np.float64(0.5965947667359811),
 'visitante': np.float64(0.15815242349426284)}

In [7]:
import joblib
MODEL_DIR = BASE_DIR / "backend" / "app" / "models"
MODEL_DIR.mkdir(parents=True, exist_ok=True)
joblib.dump(modelo, MODEL_DIR / "modelo_basico.pkl")
print("Modelo salvo em:", MODEL_DIR / "modelo_basico.pkl")


Modelo salvo em: /Users/fabrica/Documents/ia-futebol-brasil/backend/app/models/modelo_basico.pkl
